In this project, we demonstrate key data analysis and database integration techniques using a real-world loan dataset. The project offers hands-on experience in:

Data Gathering: Collecting relevant data from structured sources, such as loan information, demographic details, and financial records.

Data Cleaning and Preprocessing: Addressing common data quality issues, such as missing values and data inconsistencies, and preparing the dataset for analysis.

SQL Integration: Importing the dataset into a relational database (MySQL) and performing SQL operations, such as JOINs, SELECT, and GROUP BY to aggregate and filter data efficiently.

Exploratory Data Analysis (EDA): Using statistical and visual methods to uncover patterns, trends, and relationships in the data, including the impact of various factors on loan defaults.

This project focuses on the initial stages of a data science workflow, laying the groundwork for machine learning modeling in the next part. It helps learners understand the fundamentals of data wrangling, database interactions, and preparing data for predictive analysis.

In [1]:
# Importing Necessary Libraries

import numpy as np
import pandas as pd

In [2]:
# Loading the first 1000 rows of a credit default dataset from Kaggle for analysis
# Kaggle dataset source: https://www.kaggle.com/datasets/gustavotg/credit-default

file_path = "/Users/turmolineogi/Downloads/datacreditos.csv"
df = pd.read_csv(file_path, low_memory=False,  nrows=1000)  # Load the first 1000 rows

In [3]:
df.shape

(1000, 11)

In [4]:
# Adjusting display settings for better readability in the output
pd.set_option('display.width', 1000)

# Displaying the first few rows of the dataset to inspect the structure and initial data
df.head()

,ID,Default,Loan_Type,Gender,Age,Degree,Income,Credit_score,Loan_lenght,Signers,Citizenship
0,1,0,Car,Female,30,HS,114885,641,0,2,Citizen
1,2,0,Home,Female,43,HS,95770,534,7,2,Citizen
2,3,0,Home,Male,39,HS,94220,558,5,2,Citizen
3,4,0,Home,Male,39,College,58946,622,7,2,Citizen
4,5,0,Car,Female,42,HS,79754,702,2,1,Citizen


In [5]:
# The 'Citizenship' column does not provide any information. Removing it to clean the dataset.
df = df.drop(columns=['Citizenship'])

In [6]:
# To check how the modified dataset looks
df.head()

,ID,Default,Loan_Type,Gender,Age,Degree,Income,Credit_score,Loan_lenght,Signers
0,1,0,Car,Female,30,HS,114885,641,0,2
1,2,0,Home,Female,43,HS,95770,534,7,2
2,3,0,Home,Male,39,HS,94220,558,5,2
3,4,0,Home,Male,39,College,58946,622,7,2
4,5,0,Car,Female,42,HS,79754,702,2,1


In [7]:
# Check the dimensions of the dataset (number of rows and columns)
df.shape

(1000, 10)

In [8]:
# DATA CLEANING

In [12]:
# 1. Identify missing values in each column
print("Missing values per column:")
print(df.isnull().sum())

# Observation: No missing values found in the dataset


Missing values per column:
ID              0
Default         0
Loan_Type       0
Gender          0
Age             0
Degree          0
Income          0
Credit_score    0
Loan_lenght     0
Signers         0
dtype: int64


In [14]:
# 2. Check for duplicate rows in the dataset
duplicates = df.duplicated()

# Print the total number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

# Display duplicate rows, if any
if duplicates.sum() > 0:
    print("Duplicate rows:")
    print(df[duplicates])

# Observation from output: No duplicates found

Number of duplicate rows: 0


In [15]:
# Validating and cleaning the data types

In [16]:
# Display the data types of each column to ensure they are correct for analysis

print(df.dtypes)


ID               int64
Default          int64
Loan_Type       object
Gender          object
Age              int64
Degree          object
Income           int64
Credit_score     int64
Loan_lenght      int64
Signers          int64
dtype: object


In [ ]:
# Convert categorical columns to 'category' data type for efficient storage and analysis

categorical_columns = ['Loan_Type', 'Gender', 'Degree']
for col in categorical_columns:
    df[col] = df[col].astype('category')

In [18]:
print(df.dtypes) 

ID                 int64
Default            int64
Loan_Type       category
Gender          category
Age                int64
Degree          category
Income             int64
Credit_score       int64
Loan_lenght        int64
Signers            int64
dtype: object


In [19]:
# Converting columns to 'category' type does not change how the data is displayed

df.head()

,ID,Default,Loan_Type,Gender,Age,Degree,Income,Credit_score,Loan_lenght,Signers
0,1,0,Car,Female,30,HS,114885,641,0,2
1,2,0,Home,Female,43,HS,95770,534,7,2
2,3,0,Home,Male,39,HS,94220,558,5,2
3,4,0,Home,Male,39,College,58946,622,7,2
4,5,0,Car,Female,42,HS,79754,702,2,1


This section aims to organize the dataset into two distinct tables: borrower_info, containing demographic and credit-related details about the borrowers, and loan_details, containing information specific to loans. This separation helps in logically structuring the data for better clarity and easier analysis.

In [21]:
# Split the data into borrower_info and loan_details
borrower_info = df[['ID', 'Gender', 'Age', 'Degree', 'Income', 'Credit_score']].copy()
loan_details = df[['ID', 'Default', 'Loan_Type', 'Loan_lenght', 'Signers']].copy()

# Validate the splits by previewing the new tables

print("Borrower Info Table:")
print(borrower_info.head())

print("\nLoan Details Table:")
print(loan_details.head())


Borrower Info Table:
   ID  Gender  Age    Degree  Income  Credit_score
0   1  Female   30  HS        114885           641
1   2  Female   43  HS         95770           534
2   3  Male     39  HS         94220           558
3   4  Male     39  College    58946           622
4   5  Female   42  HS         79754           702

Loan Details Table:
   ID  Default Loan_Type  Loan_lenght  Signers
0   1        0      Car             0        2
1   2        0      Home            7        2
2   3        0      Home            5        2
3   4        0      Home            7        2
4   5        0      Car             2        1


In [22]:
# Save the tables as CSV
borrower_info.to_csv('borrower_info.csv', index=False)
loan_details.to_csv('loan_details.csv', index=False)

# Confirm the successful export of tables
print("Tables exported as CSV files for SQL integration.")


Tables exported as CSV files for SQL integration.


In this section, we establish a connection to a MySQL database using Python's SQLAlchemy library. We then import two cleaned and split datasets (borrower_info and loan_details) into MySQL as tables (BorrowerInfo and LoanDetails).

By the end of this section, the MySQL database (LoanDefaultDB) will have both tables populated with data, ready for querying and deeper insights.

In [23]:
# Importing MySQL Connector for establishing database connections
import mysql.connector

# Importing SQLAlchemy to create and manage database engine connections
from sqlalchemy import create_engine


In [24]:
# Define MySQL connection parameters

username = 'root'      # Use your MySQL username
password = 'abcdefgh'  # Use your MySQL password instead of abcdefgh
host = 'localhost'
port = '3306'

In [25]:
# Establish a connection to the MySQL server (database not specified yet)
conn = mysql.connector.connect(
    host=host,
    user=username,
    password=password
)

# Create a cursor object to execute SQL statements
cursor = conn.cursor()

# Create the LoanDefaultDB database if it doesn't already exist
cursor.execute("CREATE DATABASE IF NOT EXISTS LoanDefaultDB;")

# Switch to the LoanDefaultDB database for subsequent operations
cursor.execute("USE LoanDefaultDB;")

In [26]:
# Specify the database name to use for further operations
database = "LoanDefaultDB"

In [27]:
# # Install necessary libraries for handling data and connecting to MySQL (if not already installed)
# !pip install pandas sqlalchemy mysql-connector-python

In [28]:
from sqlalchemy import create_engine

# Create a connection string for SQLAlchemy to connect to MySQL
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)   

# Load borrower_info.csv into BorrowerInfo table
borrower_info = pd.read_csv('borrower_info.csv')
borrower_info.to_sql('BorrowerInfo', con=engine, if_exists='replace', index=False)

# Load loan_details.csv into LoanDetails table
loan_details = pd.read_csv('loan_details.csv')
loan_details.to_sql('LoanDetails', con=engine, if_exists='replace', index=False)

print("Data imported successfully!")


Data imported successfully!


In this section, we perform various SQL operations to analyze the data stored in the BorrowerInfo and LoanDetails tables. Using SQL queries, we will explore and derive insights by applying advanced techniques such as JOIN, GROUP BY, PARTITION BY, Common Table Expressions (CTEs), and aggregate functions like AVG, MAX, and MIN. These operations will help us uncover patterns, relationships, and key metrics in the dataset, enabling a deeper understanding of borrower characteristics and loan performance.

In [29]:
# Fetch all records from the BorrowerInfo table into a pandas DataFrame
query = "SELECT * FROM BorrowerInfo;"
borrower_info_df = pd.read_sql(query, con=engine)

print(borrower_info_df.head())


   ID  Gender  Age    Degree  Income  Credit_score
0   1  Female   30  HS        114885           641
1   2  Female   43  HS         95770           534
2   3  Male     39  HS         94220           558
3   4  Male     39  College    58946           622
4   5  Female   42  HS         79754           702


In [30]:
# Fetch all data from the LoanDetails table and load it into a DataFrame
query = "SELECT * FROM LoanDetails;"
loan_details_df = pd.read_sql(query, con=engine)

loan_details_df.head()

,ID,Default,Loan_Type,Loan_lenght,Signers
0,1,0,Car,0,2
1,2,0,Home,7,2
2,3,0,Home,5,2
3,4,0,Home,7,2
4,5,0,Car,2,1


In [31]:
# Exercise: Write a SQL query to combine data from two tables, BorrowerInfo and LoanDetails, 
# based on the common column ID.

query = """
SELECT 
    b.ID AS borrower_id, b.Gender, b.Age, b.Degree, b.Income, b.Credit_score,
    l.Loan_Type, l.Default, l.Loan_lenght, l.Signers
FROM BorrowerInfo b
JOIN LoanDetails l
ON b.ID = l.ID;
"""

combined_df = pd.read_sql(query, con=engine)
combined_df.head()

,borrower_id,Gender,Age,Degree,Income,Credit_score,Loan_Type,Default,Loan_lenght,Signers
0,1,Female,30,HS,114885,641,Car,0,0,2
1,2,Female,43,HS,95770,534,Home,0,7,2
2,3,Male,39,HS,94220,558,Home,0,5,2
3,4,Male,39,College,58946,622,Home,0,7,2
4,5,Female,42,HS,79754,702,Car,0,2,1


In [32]:
# Exercise: Calculate Average Income by Degree

query = """
SELECT Degree, AVG(income) AS avg_income
FROM BorrowerInfo
GROUP BY Degree
"""

avg_income_df = pd.read_sql(query, con=engine)
avg_income_df

,Degree,avg_income
0,HS,95160.2402
1,College,100800.5127
2,Gradaute,104073.5769


In [34]:
# Find the average income of borrowers by degree,
# but include the average alongside each row of data without grouping the rows.

query = """
SELECT *,
AVG(Income) OVER (PARTITION BY Degree) AS avg_income_by_degree
FROM BorrowerInfo
ORDER BY ID;
"""

partition_result = pd.read_sql(query, con=engine)
partition_result.head(10)

,ID,Gender,Age,Degree,Income,Credit_score,avg_income_by_degree
0,1,Female,30,HS,114885,641,95160.2402
1,2,Female,43,HS,95770,534,95160.2402
2,3,Male,39,HS,94220,558,95160.2402
3,4,Male,39,College,58946,622,100800.5127
4,5,Female,42,HS,79754,702,95160.2402
5,6,Female,57,Gradaute,119627,624,104073.5769
6,7,Male,41,HS,78765,594,95160.2402
7,8,Female,53,HS,123167,652,95160.2402
8,9,Female,44,HS,116175,694,95160.2402
9,10,Female,57,College,111085,586,100800.5127


Common Table Expressions (CTEs)

CTEs are temporary result sets that simplify complex queries. You can use them to structure and reuse parts of your query.

In [35]:
# Exercise: Identify borrowers whose income is above the average income for their degree.

query = """
WITH CTE_avg AS 
(SELECT *, 
AVG(Income) OVER (PARTITION BY Degree) AS avg_income_by_dep
FROM BorrowerInfo)
SELECT *
FROM CTE_avg
WHERE Income > avg_income_by_dep
ORDER BY ID;
"""
cte_result = pd.read_sql(query, con=engine)
print(cte_result.head())


   ID  Gender  Age    Degree  Income  Credit_score  avg_income_by_dep
0   1  Female   30  HS        114885           641         95160.2402
1   2  Female   43  HS         95770           534         95160.2402
2   6  Female   57  Gradaute  119627           624        104073.5769
3   8  Female   53  HS        123167           652         95160.2402
4   9  Female   44  HS        116175           694         95160.2402


In [36]:
# Exercise: Rank borrowers by income within their degree group.

query = """
WITH CTE_rank AS (
SELECT ID, Gender, Degree, Income,
RANK() OVER (PARTITION BY Degree Order BY Income DESC) AS income_rank
FROM BorrowerInfo)
SELECT * FROM CTE_rank
ORDER BY ID;

"""
ranked_borrowers = pd.read_sql(query, con=engine)
ranked_borrowers.head(10)

,ID,Gender,Degree,Income,income_rank
0,1,Female,HS,114885,81
1,2,Female,HS,95770,360
2,3,Male,HS,94220,385
3,4,Male,College,58946,234
4,5,Female,HS,79754,581
5,6,Female,Gradaute,119627,11
6,7,Male,HS,78765,598
7,8,Female,HS,123167,22
8,9,Female,HS,116175,72
9,10,Female,College,111085,67


In [37]:
# Exercise: Calculate the first 3 highest income borrowers of each group

query = """
WITH CTE_rank AS (
SELECT ID, Gender, Degree, Income,
RANK() OVER (PARTITION BY Degree Order BY Income DESC) AS income_rank
FROM BorrowerInfo)
SELECT * FROM CTE_rank
WHERE income_rank <= 3;

"""
ranked_borrowers = pd.read_sql(query, con=engine)
ranked_borrowers.head(10)

,ID,Gender,Degree,Income,income_rank
0,853,Male,College,150094,1
1,808,Male,College,147798,2
2,173,Male,College,142083,3
3,655,Female,Gradaute,135156,1
4,264,Female,Gradaute,127969,2
5,190,Male,Gradaute,126511,3
6,427,Male,HS,140787,1
7,812,Female,HS,139702,2
8,296,Female,HS,139022,3


In [38]:
# Descriptive statistics for BorrowerInfo table. 
# Key borrower trends by demographic factors such as age, gender, degree, and income.

query = """
SELECT
    AVG(Age) AS avg_age,
    MIN(Age) AS min_age,
    MAX(Age) AS max_age,
    AVG(Income) AS avg_income,
    MIN(Income) AS min_income,
    MAX(Income) AS max_income,
    AVG(Credit_score) AS avg_credit_score,
    MIN(Credit_score) AS min_credit_score,
    MAX(Credit_score) AS max_credit_score
FROM BorrowerInfo;
"""

borrower_stats = pd.read_sql(query, con=engine)
print("Borrower Summary Statistics:")
borrower_stats

Borrower Summary Statistics:


,avg_age,min_age,max_age,avg_income,min_income,max_income,avg_credit_score,min_credit_score,max_credit_score
0,42.388,24,62,96954.838,45103,150094,610.665,403,822


In [39]:
# Default rate by loan type

query = """
SELECT 
    Loan_Type,
    COUNT(*) AS total_loans,
    SUM(`Default`) AS total_defaults,
    ROUND(SUM(`Default`) * 100.0 / COUNT(*), 2) AS default_rate
FROM LoanDetails
GROUP BY Loan_Type
ORDER BY default_rate DESC;
"""

default_rate_by_loan_type = pd.read_sql(query, con=engine)
print("Default Rate by Loan Type:")
default_rate_by_loan_type

Default Rate by Loan Type:


,Loan_Type,total_loans,total_defaults,default_rate
0,Car,689,52.0,7.55
1,Home,311,11.0,3.54


In [ ]:
# Note: Important to use backticks for Default.
    
# Default is a reserved keyword in SQL, and using it without backticks would cause a syntax error.
# The backticks let the database engine know you're referring to a column named Default and not the keyword.